In [ ]:
!pip install scanpy
!pip install scikit-misc --force 
# restart runtime

In [ ]:
import numpy as np 
import pandas as pd 
import scanpy as sc 
import scanpy.tools as tl 

In [ ]:
sc.settings.verbosity = 3 
sc.settings.set_figure_params(dpi=80) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Part 1. Basic quality control and filtering

In [ ]:
# Access GSE directory and select only the lung donor, raw count matrix files

import os
lung_path = "drive/MyDrive/GSE122960_RAW/"
lung_fl = os.listdir(lung_path)
lung_donor_fl = [name for name in lung_fl if "Donor" in name and "raw" in name]
lung_donor_fl

In [ ]:
# read lung data (h5 file)

adata_lung_1 = sc.read_10x_h5(lung_path+lung_donor_fl[0])
adata = adata_lung_1

In [ ]:
adata

In [ ]:
# check if there are any duplicated gene names

dup_genes = adata.var_names[adata.var_names.duplicated()].tolist()
print(dup_genes)
print(len(dup_genes)) # there are 34 duplicated genes

# subset dataframe with all duplicated gene names -> there are multiple gene_ids for same genes
adata.var[adata.var.index.duplicated(keep=False)] 

In [ ]:
# make duplicated genes unique by appending a number string to each duplicate ('-1',' -2', etc.)

adata.var_names_make_unique()

In [ ]:
# this code is just to check how the duplicated name was renamed

dup_renamed_ls = []
for gene in adata.var_names.tolist():
  for dup in dup_genes:
    if dup in gene:
      dup_renamed_ls.append(gene) 
print(dup_renamed_ls)

In [ ]:
# show genes that yield the highest fraction of counts in each single cell, across all cells

sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
# remove cells with less than 200 genes expressed
sc.pp.filter_cells(adata, min_genes=200)

# remove genes that are expressed in less than 3 cells
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
# to filter mito genes, annotate them (start with "MT-") first 
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'

# for each cell compute fraction of counts in mito genes vs. all genes
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True) # plots fields of .obs (cells)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts') 

In [ ]:
# filter out cells with more than 3000 genes or less than 10% mito gene umi counts (cut-offs are set based on the violin and scatter plots)

adata = adata[adata.obs.n_genes_by_counts < 3000, :] 
adata = adata[adata.obs.pct_counts_mt < 10, :]
adata

## Part 2. Normalization and dimensionality reduction

In [ ]:
# normalize the counts and logarithmize

sc.pp.normalize_total(adata, target_sum=1e4) # flavor='seurat_v3' expects raw count data
sc.pp.log1p(adata) # flavor='seurat_v3' expects raw count data

In [ ]:
# https://www.kallistobus.tools/tutorials/kb_building_atlas/python/kb_analysis_0_python/
# according to the tutorial above, flavor="cell_ranger" is consistent with Seurat

sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, 
                            n_top_genes = 2000, flavor = "cell_ranger", n_bins = 20)
# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)


In [ ]:
# plot dispersions of genes vs. mean expressions of genes

sc.pl.highly_variable_genes(adata)

In [ ]:
# check how many genes were determined to be highly variable
# .value_counts() function counts a series containing counts of unique values

adata.var.highly_variable.value_counts()

In [ ]:
adata_raw = adata
adata_raw.write("drive/MyDrive/lung_results/lung_results_file_raw")
adata_raw

In [ ]:
# filter the genes, only keeping the highly variable ones

adata = adata[:, adata.var.highly_variable]
adata

In [ ]:
# regress out effects of total counts per cell and % of mito genes expressed

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
# scale each gene to unit variance. Clip values exceeding sd 10

sc.pp.scale(adata, max_value=10)

In [ ]:
# reduce the dimensionality of the data by running principal component analysis (PCA)
# PCA identifes the main axes of variation

sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# plot the first two principal components

sc.pl.pca(adata, color='SFTPB')

In [ ]:
# inspect the contribution of single PCs to the total variance in the data
# this gives an idea of how many PCs to consider when computing the neighborhood relations of cells (e.g. used in the clustering function sc.tl.louvain())

sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# save the result

adata.write("drive/MyDrive/lung_results/lung_results_file_cellranger")

In [ ]:
# compute the neighborhood graph

sc.pp.neighbors(adata, n_neighbors=20, n_pcs=30) 

In [ ]:
# ImportError: Please install the leiden algorithm: `conda install -c conda-forge leidenalg` or `pip3 install leidenalg`
!pip install leidenalg numpy python-igraph

# As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity)
sc.tl.leiden(adata) 


In [ ]:
# embed the neighborhood graph in two dimensions using UMAP

tl.paga(adata) 
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
tl.umap(adata, init_pos='paga')

In [ ]:
# plot umap

sc.pl.umap(adata, color=["leiden", 'SFTPB', 'SFTPC', 'ACE2'], wspace=0.4) # wspace is for spacing between multiple umaps. Lung dataset have a lot of clusters, legend overlaps with the next umap if you don't adjust this.
# 'SFTPB', 'SFTPC' genes are canonical markers of AT2 cells
# cluster 1, 2, 3 seem to be the AT2 cells

In [ ]:
sc.pl.violin(adata, ['SFTPB', 'SFTPC', 'ACE2'], groupby='leiden')

In [ ]:
# calculate average ACE2 expressions across AT2 cells
# adata.var.loc["ACE2":]
# "ACE2" in adata.var.index
# adata.raw.var.loc["ACE2"]

sc.get.obs_df(
    adata,
    keys = ['leiden', 'ACE2']
)


# adata.obs[''].cat.categories
# res = pd.DataFrame(columns=adata.var_names, index=adata.obs['leiden'].cat.categories)                                                                                                 
